In [1]:
import sympy as sp
import numpy as np

from sympy import I

from IPython.display import display, Latex
def pprint(S, name=""):
    """Print out a single sympy expression"""
    string = "$$" 
    if name!="": string = string + name + "=" 
    string = string + sp.latex(S) + "$$"
    return display(Latex(string))

In [2]:
dim = 4
g = sp.Matrix.diag([1, -1, -1, -1])
g

Matrix([
[1,  0,  0,  0],
[0, -1,  0,  0],
[0,  0, -1,  0],
[0,  0,  0, -1]])

In [3]:
# Pauli matrices
s = np.empty((3, 2, 2), dtype=type(sp.Rational(1)))

one = sp.Rational(1)
s[0] = np.array([
    [0, one],
    [one, 0]
])
s[1] = np.array([
    [0, -I],
    [I, 0]
])
s[2] = np.array([
    [one, 0],
    [0, -one]
])

In [4]:
# Dirac matricies

Oh = np.zeros((2, 2), dtype=type(sp.Rational))
eye = np.identity(2, dtype=type(sp.Rational))

γ = np.empty((dim, dim, dim), dtype=type(sp.Rational(1)))
γ[0] = np.block([
    [eye, Oh],
    [Oh, -eye]
])

for i in range(3):
    γ[i + 1] = np.block([
        [Oh, s[i]],
        [-s[i], Oh]
    ])
    
γ = sp.Array(γ)
γ

[[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, -1, 0], [0, 0, 0, -1]], [[0, 0, 0, 1], [0, 0, 1, 0], [0, -1, 0, 0], [-1, 0, 0, 0]], [[0, 0, 0, -I], [0, 0, I, 0], [0, I, 0, 0], [-I, 0, 0, 0]], [[0, 0, 1, 0], [0, 0, 0, -1], [-1, 0, 0, 0], [0, 1, 0, 0]]]

In [5]:
γ5 = I
for mat in γ:
    γ5 *= sp.Matrix(mat)
γ5

Matrix([
[0, 0, 1, 0],
[0, 0, 0, 1],
[1, 0, 0, 0],
[0, 1, 0, 0]])

In [6]:
from sympy import tensorproduct as tp
from sympy import tensorcontraction as tc

def prod(A, v):
    return tc(tp(A, v), (1, 2))

def dot(v1, v2):
    return tc(tp(v1, v2), (0, 1))

def sum(lst):
    """ Must overwrite built in to sum sybols, for some reason """
    s = 0
    for l in lst:
        s += l
    return s

In [7]:
def slash(a):
    slasha = 0 * γ[0]
    for i in range(dim):
        slasha += γ[i] * g[i, i] * a[i]
    return slasha


def Tr(lst):
    global γ
    
    free_indx = [γ == a for a in lst]
    A = lst[0]
    for i, a in enumerate(lst[1:]):
        A = tp(A, a)
        i1 = sum(free_indx[:i+1])
        i2 = sum(free_indx[:i+2])
        A = tc(A, (1 + i1, i2 + 2))

    last = len(A.shape) - 1
    first = free_indx[0] + 0
    return tc(A, (first, last))

# Mott scattering

The first Feynmann diagram for electron-muon scattering gives, after using Casimirs trick, the amplitude
$$
\langle | \mathcal{M} |^2 \rangle = \frac{g_e^4}{4 (p_1 - p_4)^2} \mathrm{Tr}\left[ \gamma^\mu (\gamma_\nu p_3^\nu - m) \gamma^\sigma (\gamma_\rho p_1^\rho - m) \right] \mathrm{Tr}\left[ \gamma_\mu (\gamma_\nu p_2^\nu - m) \gamma_\sigma (\gamma_\rho p_4^\rho - m) \right]
$$

In [8]:
num_vec = 4
p = sp.Array([
    sp.symbols("p^{" + str(i) + "}_" + str(j)) for j in range(num_vec) 
    for i in range(dim)
]).reshape(num_vec, dim)

p_ = sp.Array([prod(g, p[i]) for i in range(num_vec)])

In [9]:
m1, m2 = sp.symbols("m_1, m_2")
Id = sp.Array(sp.eye(4))
slash(p[0]) - m1 * Id

[[-m_1 + p^{0}_0, 0, -p^{3}_0, -p^{1}_0 + I*p^{2}_0], [0, -m_1 + p^{0}_0, -p^{1}_0 - I*p^{2}_0, p^{3}_0], [p^{3}_0, p^{1}_0 - I*p^{2}_0, -m_1 - p^{0}_0, 0], [p^{1}_0 + I*p^{2}_0, -p^{3}_0, 0, -m_1 - p^{0}_0]]

In [10]:
M1 = Tr((γ, slash(p[0]) - m1 * Id, γ, slash(p[2]) - m1 * Id))
M2 = Tr((γ, slash(p[1]) - m2 * Id, γ, slash(p[3]) - m2 * Id))
M2_ = prod(g, prod(M2, g))

In [11]:
ge = sp.symbols("g_e")
coeff = ge**4 /(4 * dot(p[0] - p[2], p_[0] - p_[2])**2)
sp.simplify(coeff)

g_e**4/(4*(-(p^{0}_0 - p^{0}_2)**2 + (p^{1}_0 - p^{1}_2)**2 + (p^{2}_0 - p^{2}_2)**2 + (p^{3}_0 - p^{3}_2)**2)**2)

In [12]:
M = coeff * tc(tc(tp(M1, M2_), (0, 3)), (0, 1))
sp.simplify(M)

g_e**4*(-2*(-(p^{1}_0 - I*p^{2}_0)*(p^{1}_2 - I*p^{2}_2) + (p^{1}_0 + I*p^{2}_0)*(p^{1}_2 + I*p^{2}_2))*(-(p^{1}_1 - I*p^{2}_1)*(p^{1}_3 - I*p^{2}_3) + (p^{1}_1 + I*p^{2}_1)*(p^{1}_3 + I*p^{2}_3)) + 2*(p^{3}_0*(m_1 - p^{0}_2) - p^{3}_0*(m_1 + p^{0}_2) + p^{3}_2*(m_1 - p^{0}_0) - p^{3}_2*(m_1 + p^{0}_0))*(-p^{3}_1*(m_2 - p^{0}_3) + p^{3}_1*(m_2 + p^{0}_3) - p^{3}_3*(m_2 - p^{0}_1) + p^{3}_3*(m_2 + p^{0}_1)) - 2*(-p^{3}_0*(p^{1}_2 - I*p^{2}_2) + p^{3}_0*(p^{1}_2 + I*p^{2}_2) - p^{3}_2*(p^{1}_0 - I*p^{2}_0) + p^{3}_2*(p^{1}_0 + I*p^{2}_0))*(-p^{3}_1*(p^{1}_3 - I*p^{2}_3) + p^{3}_1*(p^{1}_3 + I*p^{2}_3) - p^{3}_3*(p^{1}_1 - I*p^{2}_1) + p^{3}_3*(p^{1}_1 + I*p^{2}_1)) + 2*(p^{3}_0*(p^{1}_2 - I*p^{2}_2) + p^{3}_0*(p^{1}_2 + I*p^{2}_2) + p^{3}_2*(p^{1}_0 - I*p^{2}_0) + p^{3}_2*(p^{1}_0 + I*p^{2}_0))*(p^{3}_1*(p^{1}_3 - I*p^{2}_3) + p^{3}_1*(p^{1}_3 + I*p^{2}_3) + p^{3}_3*(p^{1}_1 - I*p^{2}_1) + p^{3}_3*(p^{1}_1 + I*p^{2}_1)) + (-2*p^{3}_0*p^{3}_2 + (m_1 - p^{0}_0)*(m_1 + p^{0}_2) + (m_1 + p^{

What a horrible mess! However, we may insert give som explisit values for the momenta. For example, we can set
$$
    p^{(1)} =  [E, p, 0, 0] \\
    p^{(2)} = [M, 0, 0, 0] \\
    p^{(3)} = [E, p \cos(\theta), p \sin(\theta), 0]\\
    p^{(4)} = [M, 0, 0, 0] \\
$$

In [13]:
E, P, theta = sp.symbols("E, p, \\theta")

p_vals = sp.Array([
    [E, P, 0, 0],
    [m2, 0, 0, 0],
    [E, P * sp.cos(theta), P * sp.sin(theta), 0],
    [m2, 0, 0, 0]
])
p_vals

[[E, p, 0, 0], [m_2, 0, 0, 0], [E, p*cos(\theta), p*sin(\theta), 0], [m_2, 0, 0, 0]]

In [14]:
v = p_vals[0] - p_vals[2]
v_ = prod(g, v)
sp.simplify(dot(v, v_)**2)

p**4*(2 - 2*cos(\theta))**2

In [15]:
Ms = M
for i in range(num_vec):
    for j in range(dim):
        Ms = Ms.subs(p[i, j], p_vals[i, j])
    
sp.simplify(Ms.subs(E, sp.sqrt(m1**2 + P**2)))

2*g_e**4*m_2**2*(2*m_1**2 + p**2*cos(\theta) + p**2)/(p**4*(cos(\theta) - 1)**2)